In [2]:
import numpy as np
import cvxportfolio as cvx
from universes import NDX100 as UNIVERSE

In [4]:


# times
START = '2019-01-01'
END = None # today

# Currently (~2024) shorting large cap US stocks costs about this,
# in annualized percentages
BORROW_FEES = 0.25

# We set the bid-ask spreads at 5 basis points
SPREAD = 5E-4

# This is the b multiplier of the (3/2) power term in TransactionCost
MARKET_IMPACT = 1.

policy = cvx.SinglePeriodOptimization(
    objective=cvx.ReturnsForecast()
        - cvx.Gamma() * cvx.FactorModelCovariance(num_factors=10)
        - cvx.Gamma() * cvx.TransactionCost(a=SPREAD/2, b=MARKET_IMPACT)
        - cvx.Gamma() * cvx.HoldingCost(short_fees=BORROW_FEES),

    constraints = [
        cvx.DollarNeutral(), cvx.MarketNeutral(), cvx.LeverageLimit(7)],
    # this solver is somewhat more robust than ECOS, but less efficient
    solver='CLARABEL',
    # this is a CVXPY compilation flag that disables a feature that is very
    # useful (cache a semi-compiled problem) but its implementation scales
    # badly with the problem size; if you increase number of factors or
    # universe size, you may have to uncomment the next line
    # ignore_dpp=True,
)

print("Universe:", UNIVERSE)
simulator = cvx.MarketSimulator(
    universe=UNIVERSE,
    costs = [
        cvx.TransactionCost(a=SPREAD/2, b=MARKET_IMPACT),
        cvx.HoldingCost(short_fees=BORROW_FEES)])

# automatic hyper-parameter optimization (by greedy grid search)
simulator.optimize_hyperparameters(
    policy, start_time=START, end_time=END,
    objective='sharpe_ratio')

print('Optimized policy hyper-parameters:')
print(policy)

# back-test the policy with optimized hyper-parameters
result = simulator.backtest(policy, start_time=START, end_time=END)

print("Optimized policy back-test result:")
print(result)

# plot
result_figure = result.plot()

# check that back-tested returns of the strategy are uncorrelated with the
# market benchmark
print("back testing")
market_benchmark_returns = simulator.backtest(
    cvx.MarketBenchmark(), start_time=START, end_time=END).returns

print('Correlation of strategy returns with benchmark:')
print(np.corrcoef(result.returns, market_benchmark_returns)[0, 1])

Universe: ['AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'ANSS', 'ASML', 'AVGO', 'AZN', 'BIIB', 'BKNG', 'BKR', 'CCEP', 'CDNS', 'CDW', 'CEG', 'CHTR', 'CMCSA', 'COST', 'CPRT', 'CRWD', 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTSH', 'DASH', 'DDOG', 'DLTR', 'DXCM', 'EA', 'EXC', 'FANG', 'FAST', 'FTNT', 'GEHC', 'GFS', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX', 'ILMN', 'INTC', 'INTU', 'ISRG', 'KDP', 'KHC', 'KLAC', 'LIN', 'LRCX', 'LULU', 'MAR', 'MCHP', 'MDB', 'MDLZ', 'MELI', 'META', 'MNST', 'MRNA', 'MRVL', 'MSFT', 'MU', 'NFLX', 'NVDA', 'NXPI', 'ODFL', 'ON', 'ORLY', 'PANW', 'PAYX', 'PCAR', 'PDD', 'PEP', 'PYPL', 'QCOM', 'REGN', 'ROP', 'ROST', 'SBUX', 'SIRI', 'SNPS', 'TEAM', 'TMUS', 'TSLA', 'TTD', 'TTWO', 'TXN', 'VRSK', 'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL', 'ZS']
Updating data................

.....................................................................................
iteration 0
Current objective:
0.6720607535018006
iteration 1
Current objective:
0.6876009243159951
iteration 2
Current objective:
0.6952564661935553
iteration 3
Current objective:
0.7031524342445792
iteration 4
Current objective:
0.7150216346010108
iteration 5
Current objective:
0.7247588216259265
iteration 6
Current objective:
0.7307022451726265
iteration 7
Current objective:
0.7346783953944651
iteration 8
Current objective:
0.740498860819599
iteration 9
Current objective:
0.7428750794281525
iteration 10
Current objective:
0.7461327811438719
iteration 11
Current objective:
0.7494839381985818
iteration 12
Current objective:
0.7513578385220537
iteration 13
Current objective:
0.7514602885225721
iteration 14
Current objective:
0.7514972054691452
iteration 15
Current objective:
0.7515435798993501
iteration 16
Current objective:
0.7515766968840812


KeyboardInterrupt: 